# भाग 2: Syft Keras सह सुरक्षित मॉडेल सर्व्हिंग

आता आपल्याकडे सामान्य Keras असलेले एक प्रशिक्षित मॉडेल आहे, आपण काही खाजगी अंदाज देण्यासाठी तयार आहात. हे आपण Syft Keras वापरुन करू शकतो.

हे मॉडेल सुरक्षित आणि सेवा देण्यासाठी, आपल्याला तीन TFEWorkers (सर्व्हर) आवश्यक असतील. याचे कारण असे की हुड अंतर्गत TF एनक्रिप्टेड केलेले [मल्टी-पार्टी कंप्यूटेशन (MPC)](https://en.wikipedia.org/wiki/Secure) नावाचे एनक्रिप्शन तंत्र वापरते. मॉडेल वजनाची आणि इनपुट डेटाची शेअर्समध्ये विभागणी करणे, त्यानंतर प्रत्येक सर्व्हरला एक हिस्सा वेगवेगळ्या सर्व्हर्सवर पाठविण्याची कल्पना आहे. मुख्य गुणधर्म अशी आहे की जर आपण एका सर्व्हरवरील शेअर्सकडे पाहिले तर ते मूळ मूल्याबद्दल काहीही सांगत नाही (इनपुट डेटा किंवा मॉडेल वजन).

आम्ही मागील नोटबुक प्रमाणेच एक Syft Keras मॉडेल परिभाषित करू. तथापि, एक युक्ती आहेः हे मॉडेल स्थापित करण्यापूर्वी, आम्ही `hook = sy.KerasHook (tf.keras)` चालवू. हे केरास अनुक्रमिक वर्गामध्ये तीन महत्त्वाच्या नवीन पद्धती जोडेल:
- `share`: गुपित सामायिकरणातून त्याचे मॉडेल सुरक्षित करेल; डीफॉल्टनुसार, tf  एन्क्रिप्टेडकडून SecureNN प्रोटोकॉल वापरुन प्रत्येक तीन TFEWorkers प्रत्येकात ते गुप्तपणे आपले मॉडेल सामायिक करेल. सर्वात महत्त्वाचे म्हणजे, ते एन्क्रिप्टेड डेटावर अंदाज करण्याची क्षमता जोडेल.
- `serve`: हे कार्य एक सर्व्हिंग रांग लाँच करेल, जेणेकरून TFEWorkers  सुरक्षित मॉडेलवर बाह्य ग्राहकांकडील अंदाज विनंत्या स्वीकारू शकतील.
- `shutdown_workers`: एकदा आपण खाजगी अंदाज दिल्यास आपण हे कार्य चालवून आपले मॉडेल बंद करू शकता. आपण प्रत्येक कामगार व्यक्तिचलितपणे व्यवस्थापित करणे निवडले असल्यास ते आपल्याला सर्व्हर प्रक्रिया बंद करण्यासाठी व्यक्तिचलितपणे निर्देशित करते.

आपण MPC अधिक जाणून घेऊ इच्छित असल्यास आपण हा उत्कृष्ट [ब्लॉग](https://mortendahl.github.io/2017/04/17/pStreet-deep-learning-with-mpc/) वाचू शकता.

अनुवादक/संपादक:
- Krunal Kshirsagar - Twitter: [@krunal_wrote](https://twitter.com/krunal_wrote) - Github: [@Noob-can-Compile](https://github.com/Noob-can-Compile)


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

import syft as sy
hook = sy.KerasHook(tf.keras)

## मॉडेल

आपण पाहू शकता की आम्ही `batch_input_shape` प्रदान केल्याशिवाय आम्ही पूर्वीप्रमाणेच अगदी जवळजवळ समान मॉडेलची व्याख्या केली आहे. हे TF एन्क्रिप्टेडला पूर्वनिर्धारित टेन्सर आकारांद्वारे सुरक्षित संगणनास अधिक अनुकूलित करण्यास अनुमती देते. या MNIST डेमोसाठी आम्ही इनपुट डेटा पाठवू (1, 28, 28, 1). आम्ही हे ऑपरेशन MPC वापरून करण्यासाठी क्लिष्ट आहे कारण softmax ऐवजी logit परत, आणि आम्ही अंदाज विनंत्या सेवा करण्यासाठी आपल्याला त्याची आवश्यकता नाही.

In [2]:
num_classes = 10
input_shape = (1, 28, 28, 1)

In [3]:
model = Sequential()

model.add(Conv2D(10, (3, 3), batch_input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, name="logit"))

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


### पूर्व-प्रशिक्षित वजन लोड करा

`load_weights` सह आपण आपल्या मॉडेलचे प्रशिक्षण घेतल्यानंतर आपण जतन केलेले वजन(weights) सहजपणे लोड करू शकता.

In [4]:
pre_trained_weights = 'short-conv-mnist.h5'
model.load_weights(pre_trained_weights)

### कामगार लाँच करा

आता खाजगी अंदाज घेण्यासाठी TF Encrypted कडून आवश्यक TFEWorkers (`alice`,` bob`, और `carol`) तयार करूया. प्रत्येक TFEWorker साठी, आपल्याला फक्त एक होस्ट निर्दिष्ट करावा लागेल. त्यानंतर आम्ही या कामगारांना क्लस्टरमध्ये एकत्र करतो.

हे कामगार [TensorFlow सर्व्हर](https://www.tensorflow.org/api_docs/python/tf/distribute/Server) चालवतात, जे आपण एकतर व्यक्तिचलितरित्या व्यवस्थापित करू शकता (`AUTO = False`) किंवा कामगारांना आपल्यासाठी व्यवस्थापित करण्यास सांगू शकता (`AUTO = True`). त्या व्यक्तिचलितरित्या व्यवस्थापित करणे निवडल्यास, आपल्याला खाली असलेल्या `cluster.start()` वर कॉल केल्यानंतर प्रत्येक कामगारांच्या होस्ट डिव्हाइसवर टर्मिनल आदेश चालविण्यास सूचित केले जाईल. जर सर्व कामगार एका डिव्हाइसवर (उदा. `localhost`) वर होस्ट केलेले असतील तर आपण Syft आपोआप कामगारांचे TensorFlow सर्व्हर व्यवस्थापित करू शकता.

In [5]:
AUTO = False

alice = sy.TFEWorker(host='localhost:4000', auto_managed=AUTO)
bob = sy.TFEWorker(host='localhost:4001', auto_managed=AUTO)
carol = sy.TFEWorker(host='localhost:4002', auto_managed=AUTO)

cluster = sy.TFECluster(alice, bob, carol)
cluster.start()

INFO:tf_encrypted:If not done already, please launch the following command in a terminal on host localhost:4000: 'python -m tf_encrypted.player --config /var/folders/mh/7hh_sz1d3532_k9w4kcmnt8c0000gn/T/tfe.config server0'
This can be done automatically in a local subprocess by setting `auto_managed=True` when instantiating a TFEWorker.

INFO:tf_encrypted:If not done already, please launch the following command in a terminal on host localhost:4001: 'python -m tf_encrypted.player --config /var/folders/mh/7hh_sz1d3532_k9w4kcmnt8c0000gn/T/tfe.config server1'
This can be done automatically in a local subprocess by setting `auto_managed=True` when instantiating a TFEWorker.

INFO:tf_encrypted:If not done already, please launch the following command in a terminal on host localhost:4002: 'python -m tf_encrypted.player --config /var/folders/mh/7hh_sz1d3532_k9w4kcmnt8c0000gn/T/tfe.config server2'
This can be done automatically in a local subprocess by setting `auto_managed=True` when instantiati

### वजन सामायिक करुन मॉडेलला सुरक्षित करा

`sy.KerasHook (tf.keras)` चे आभार मानून आपण आपल्या मॉडेलला TF एन्क्रिप्टेड Keras मॉडेलमध्ये रूपांतरित करण्यासाठी `share` पद्धत कॉल करू शकता.

जर आपल्याला सर्व्हर व्यक्तिचलितपणे व्यवस्थापित करण्यास सांगितले असेल तर ते सर्व लाँच होईपर्यंत हे चरण पूर्ण होणार नाही. लक्षात ठेवा की आपली फायरवॉल पायथनला येणारे कनेक्शन स्वीकारण्यास विचारू शक्तो.

In [6]:
model.share(cluster)

INFO:tf_encrypted:Starting session on target 'grpc://localhost:4000' using config graph_options {
}



### मॉडेल सर्व्ह करावे

परिपूर्ण! आता `model.serve` कॉल करून, आपले मॉडेल काही खाजगी अंदाज देण्यासाठी सज्ज आहे. आपण मॉडेलद्वारे दिल्या गेलेल्या पूर्वानुमान विनंत्यांच्या संख्येवर मर्यादा सेट करण्यासाठी `num_requests` सेट करू शकता; निर्दिष्ट न केल्यास मॉडेल व्यत्यय येईपर्यंत दिला जाईल.

In [7]:
model.serve(num_requests=3)

Served encrypted prediction 1 to client.
Served encrypted prediction 2 to client.
Served encrypted prediction 3 to client.


आपण काही खाजगी अंदाज मागण्यासाठी पार्ट 13 सी नोटबुकवर जाण्यास तयार आहात.

### क्लिनअप!

एकदा आपली विनंती मर्यादा झाली की, मॉडेल्स यापुढे सर्व्हिसिंग विनंत्यांसाठी उपलब्ध नसतील, परंतु तरीही वरील तीन कामगारांमध्ये ते सामायिक आहे. आपण खाली सेल चालवून कामगारांना मारू शकता.

**अभिनंदन** भाग 13 बी पूर्ण केल्याबद्दल: सिफ्ट केरास आणि टीएफईसह सुरक्षित वर्गीकरण!

In [8]:
model.stop()
cluster.stop()

if not AUTO:
    process_ids = !ps aux | grep '[p]ython -m tf_encrypted.player --config' | awk '{print $2}'
    for process_id in process_ids:
        !kill {process_id}
        print("Process ID {id} has been killed.".format(id=process_id))

INFO:tf_encrypted:Please terminate the process on host 'localhost:4000'.
INFO:tf_encrypted:Please terminate the process on host 'localhost:4001'.
INFO:tf_encrypted:Please terminate the process on host 'localhost:4002'.


Process ID 12470 has been killed.
Process ID 12474 has been killed.
Process ID 12464 has been killed.
